In [1]:
import os
import nway
import json

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from dandi import dandiapi
from PIL import Image
from pynwb import NWBHDF5IO


In [286]:
dandiset_id = "000336"
dandi_filepath = "sub-1170622509/sub-1170622509_ses-1194683535-1194752574-_ophys.nwb"

authenticate = True
dandi_api_key = os.environ["dandi_api_key"]

In [287]:
if authenticate:
    client = dandiapi.DandiAPIClient(token=dandi_api_key)
else:
    client = dandiapi.DandiAPIClient()
my_dandiset = client.get_dandiset(dandiset_id)

filename = dandi_filepath.split("/")[-1]
filepath = f"./{filename}"
file = my_dandiset.get_asset_by_path(dandi_filepath)

In [288]:
# # this may take awhile, especially if the file to download is large
# file.download(filepath)

# print(f"Downloaded file to {filepath}")

Downloaded file to ./sub-1170622509_ses-1194683535-1194752574-_ophys.nwb


In [393]:
data_dir = "../../../data/dendrite_coupling"
io = NWBHDF5IO(f"{data_dir}/sub-1168633075_ses-1193555033-1193675745-_ophys.nwb", mode="r", load_namespaces=True)
nwb1 = io.read()

io = NWBHDF5IO(f"{data_dir}/sub-1170622509_ses-1194683535-1194752574-_ophys.nwb", mode="r", load_namespaces=True)
nwb2 = io.read()

In [394]:
nwbs = [nwb1, nwb2]

### Convert to Nwaymatching Input File

In [395]:
def make_roi_entries(roi_table):
    roi_entries = []
    for roi_idx in range(len(roi_table)):
        x = int(roi_table["x"][roi_idx])
        y = int(roi_table["y"][roi_idx])
        height = int(roi_table["height"][roi_idx])
        width = int(roi_table["width"][roi_idx])
        
        mask_subimage = roi_table["image_mask"][roi_idx][y:y+height,x:x+width]
        
        roi_entry = {
            "id": int(roi_table["id"][roi_idx]),
            "valid": bool(roi_table["valid_roi"][roi_idx]),
            "x": x,
            "y": y,
            "z": 1,
            "width": width,
            "height": height,
            # convert 2D array to 2D list of bools
            "mask_matrix": [[bool(elem) for elem in row] for row in mask_subimage]
        }
        roi_entries.append(roi_entry)
    return roi_entries

In [396]:
def normalize_to_16b(image_array):
    mn = image_array.min()
    mx = image_array.max()
    mx -= mn
    image_array = ((image_array - mn)/mx) * (2**16)
    return image_array.astype(np.uint16)

In [353]:
def nwb_to_experiment_json(nwb):
    # save image here
    grayscale_image = np.array(nwb.processing["ophys"]["images"]["average_image"])
    image_array = normalize_to_16b(np.array(grayscale_image))
    image_file = Image.fromarray(image_array)
    image_file = image_file.convert("L")
    image_file.save(f"./{nwb.identifier}_image.png")

    roi_table = nwb.processing["ophys"]["image_segmentation"]["cell_specimen_table"]
    
    experiment = {
        "id": nwb.identifier, 
        "ophys_average_intensity_projection_image": f"./{nwb.identifier}_image.png",
        "cell_rois": make_roi_entries(roi_table)
    }

    return experiment

In [354]:
input_exps = {
    "experiment_containers": {
        "ophys_experiments": []
    }
}

for nwb in nwbs:
    experiment = nwb_to_experiment_json(nwb)
    input_exps["experiment_containers"]["ophys_experiments"].append(experiment)

with open("input.json", "w") as f:
    json.dump(input_exps, f, indent=2)

In [355]:
!python -m nway.nway_matching --input_json input.json --output_json test_output/output.json --output_dir test_output

INFO:NwayMatching:NWAY_COMMIT_SHA None
INFO:NwayMatching:Nway matching version 0.6.0
INFO:PairwiseMatching:Matching 1193675745 to 1194752574
INFO:PairwiseMatching:Matching 1193675745 to 1194752574: best registration was ['Crop', 'ECC']
C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\nway\utils.py:48: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
  df = df.sort_index(0)
C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\nway\utils.py:49: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
  df = df.sort_index(1)
INFO:NwayMatching:registration success(1) or failure (0):
   0  1
0  1  1
1  1  1
id map{
  "0": 1193675745,
  "1": 1194752574
}
Traceback (most recent call last):
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_g